# Isolated Limb Perfusion to OMOP

For the treatments, we follow the mapping in https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=1595135203#gid=1595135203

This is mainly mapped to episode and procedure occurrence

We need the procedure ocncept id, using 4019656

In [ ]:
import pandas as pd
import psycopg2
import random
import numpy as np
from uuid import uuid4
from datetime import datetime, timedelta
import mysql.connector

In [ ]:
isolated_limb_perfusion_IDEA4RC = pd.read_csv("./IDEA4RC-data/isolatedLimbPerfusionIDEA4RC.csv")
isolated_limb_perfusion_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()
cur = conn.cursor()

### Isolated Limb Perfusion to Episode

In [ ]:
df_tables = isolated_limb_perfusion_IDEA4RC
sql = """
INSERT INTO EPISODE (episode_id,person_id, episode_start_date, episode_end_date, episode_concept_id,episode_type_concept_id, parent_episode_id)
VALUES (%s,%s,%s,%s,%s,%s,%s);
"""
queryPerson="""
    SELECT e.patient_id FROM cancer_episode e
    INNER JOIN episode_event v ON e.id=v.cancer_episode
    INNER JOIN systemic_treatment st ON st.episode_event=v.id
    WHERE st.id=%s;
"""    

queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s;
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1;
"""
df_tables['idThisEvent'] = None
for idx, row in df_tables.iterrows():
    curIDEA.execute(queryPerson, (row['id']))
    res=curIDEA.fetchone()
    person_id=res[0]
    curIDEA.execute(queryDate, (row['episode_event']))
    res=curIDEA.fetchone()
    dateAux=res
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    parent_episode_id=cur.fetchone()
    episode_start_date=res
    newId=datetime.now().strftime('%Y%m%d%H%M%S')
    row['idThisEvent']=newId
    cur.execute(sql,(newId,person_id, row['start_date'], row['end_date'],row['treatment_response'],row['treatment_response'],parent_episode_id))
    conn.commit()

### Isolated Limb Perfusion to Procedure  Ocurrence

In [ ]:
queryEVPO="""
    INSERT INTO omopcdm.episode_event (episode_id,event_id,episode_event_field_concept_id)
    VALUES (%s, %s, %s);
"""

sqlProcedure="""
    INSERT INTO omopcdm.procedure_ocurrence (procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_end_date,procedure_type_concept_id,provider_id)
    VALUES (%s, %s, %s,%s,%s);
"""

getProvider_id="""
    SELECT provider_id FROM provider WHERE care_site_id IN(SELECT care_site_id FROM care_site WHERE care_site_name=%s);
"""

def toTableEpisodeEventProcedureOcurrence(idEpisode , idProcedureOcurrence):
    cur.execute(queryEVPO,(idEpisode,idProcedureOcurrence,1147082)) #1147082 or 1147810, not clear
    conn.commit()

for idx, row in df_tables.iterrows():
    cur.execute(queryPerson, (row['episode_event']))
    res=cur.fetchone()
    person_id=res
    procedure_concept_id=4019656
    procedure_date=row['start_date']
    procedure_end_date=row['end_date']
    procedure_type_concept_id=32879 #Still dont know what to do with this one
    newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
    toTableEpisodeEventProcedureOcurrence(row['idThisEvent'],newId)
    cur.execute(getProvider_id, (row['care_site']))
    res=cur.fetchone()
    provider_id=res[0]
    cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_end_date,procedure_type_concept_id,provider_id))
    conn.commit()